In [1]:
using PCT

┌ Info: Precompiling PCT [ef708a43-c8a3-43f4-8f65-1f04ee4c5bb0]
└ @ Base loading.jl:1664


In [18]:
f = @pct begin

@domain begin
    name=Na
    base=I
    lower=-N
    upper=N
end

@space begin
    name=M
    type=(I, I) -> R
    symmetries=((1,2),)
end
@space begin
    name=V
    type=(I,) -> R
end

# (y::V) -> (j::I) -> ((x(i)::V) -> x(i))(y(j))
# (y::V) -> (j::I) -> pullback((x(i)::V) -> x(j) * y(j))
# pullback((x::R) -> x * x + x)
pullback((x::V) -> sum((i::Na,j::Na), x(i) * x(i)))
end

𝒫((x) -> 
    ∑((i), ∑((j), x(i)*x(i))))

In [17]:
@macroexpand @pct begin
    pullback((x::V) -> sum((i,j::Na), x(i) * x(i)))
end

quote
    #= /Users/likangbo/kaer_morhen/projects/QLD/PCT/src/syntax.jl:10 =#
    make_node!(Pullback, make_node!(Map, make_node!(PCTVector, make_node!(Var, :x, type = V)), make_node!(Sum, make_node!(Var, :i), make_node!(Var, :j, type = Na), make_node!(Mul, make_node!(PCTVector, make_node!(PrimitiveCall, make_node!(Var, :x), make_node!(PCTVector, make_node!(Var, :i))), make_node!(PrimitiveCall, make_node!(Var, :x), make_node!(PCTVector, make_node!(Var, :i))))))))
end

In [19]:
f = inference(f)

𝒫((x) -> 
    ∑((i), ∑((j), x(i)*x(i))))

In [20]:
e_f = evaluate(f)

𝒫((x) -> 
    ∑((i), ∑((j), x(i)*x(i))))

In [21]:
println(verbose(e_f))

Pullback((x::[I->R]->
    ∑((i::Na),
        ∑((j::Na),
            (*
                x::[I->R](i::Na)::R,
                x::[I->R](i::Na)::R
            )::R
        )::R
    )::R
::[[I->R]->R])::[[I->R]×R->[I->R]]


In [23]:
p = evaluate_pullback(e_f)

(x, 𝒦) -> 
    (i_1) -> 
        ((x(i_1), 𝒦) -> 
            ∑((i), ((x(i_1), 𝒦) -> 
                ∑((j), ((x(i_1), 𝒦) -> 
                    (((x(i_1), 𝒦) -> 
                        δ(i_1, i, 𝒦))(x(i_1), x(i)*𝒦)+((x(i_1), 𝒦) -> 
                        δ(i_1, i, 𝒦))(x(i_1), x(i)*𝒦)))(x(i_1), 𝒦)))(x(i_1), 𝒦)))(x(i_1), 𝒦)

In [24]:
e_p = evaluate(p)

(x, 𝒦) -> 
    (i_1) -> 
        ∑((i), ∑((j), (δ(i_1, i, x(i)*𝒦)+δ(i_1, i, x(i)*𝒦))))

In [26]:
println(verbose(e_p))

(x::[I->R], 𝒦::R->
    (i_1::I->
        ∑((i::I),
            ∑((j::I),
                (+
                    δ(i_1::I, i::I,
                        (*
                            x::[I->R](i::I)::R,
                            𝒦::R
                        )::R)::R,
                    δ(i_1::I, i::I,
                        (*
                            x::[I->R](i::I)::R,
                            𝒦::R
                        )::R)::R
                )::R
            )::R
        )::R
    ::[I->R]
::[[I->R]×R->[I->R]]


In [25]:
 delta_reduce(e_p)

(x, 𝒦) -> 
    (i_1) -> 
        ∑((j), (x(i_1)*𝒦+x(i_1)*𝒦))

In [49]:
Meta.dump(Meta.parse(
    "
@range begin
    name=M
    base=I
    lower=N
    upper=N
end"), maxdepth=20)

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @domain
    2: LineNumberNode
      line: Int64 2
      file: Symbol none
    3: Expr
      head: Symbol block
      args: Array{Any}((8,))
        1: LineNumberNode
          line: Int64 3
          file: Symbol none
        2: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol name
            2: Symbol M
        3: LineNumberNode
          line: Int64 4
          file: Symbol none
        4: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol base
            2: Symbol I
        5: LineNumberNode
          line: Int64 5
          file: Symbol none
        6: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol lower
            2: Symbol N
        7: LineNumberNode
          line: Int64 6
          file: Symbol none
        8: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol upper
 

In [26]:
using PCT
f = @pct begin
@domain begin
    name=M
    base=I
    lower=-N
    upper=N
end
    
(x::M) -> x+1
end 

(x) -> 
    (x+1)

In [148]:
f = @pct begin
@space begin
    name=M
    type=(I, I) -> R
    symmetries=(((2,1),:id),)
end

        
(A::M) -> (i::I, j::I) -> A(i, j) + A(j, i)
end

(A) -> 
    (i, j) -> 
        (A(i, j)+A(j, i))

In [149]:
f = inference(f)

(A) -> 
    (i, j) -> 
        (A(i, j)+A(j, i))

In [150]:
println(verbose(fc(fc(f))))

(+
    A::[I×I->R](i::I, j::I)::R,
    A::[I×I->R](j::I, i::I)::R
)::R


In [151]:
last(neighbors(fc(fc(f))).nodes)

(A(i, j)+A(i, j))

In [152]:
neighbors(last(neighbors(fc(fc(f))).nodes))

(A(j, i)+A(i, j))
(A(i, j)+A(j, i))
(A(i, j)*(1+1))

In [58]:
v = make_node!(PCTVector, make_node!(Var, :i), make_node!(Var, :j))

i, j

In [59]:
v[1]

i

In [82]:
v[1:2] |> typeof

PCTVector

In [65]:
context = @macroexpand @pct begin
@domain begin
    name=M
    base=I
    lower=-N
    upper=N
end
end 

quote
    #= /Users/likangbo/kaer_morhen/projects/QLD/PCT/src/syntax.jl:10 =#
    M = Domain(I(), mul(make_node!(Constant, -1), make_node!(Var, :N)), make_node!(Var, :N), Dict(:name => :M))
end

In [74]:
f = @pct (A::M) -> A context

(A) -> 
    A

In [75]:
println(verbose(inference(f)))

(A::M->
    A::M
::[M->M]


In [31]:
a = [1,2]
a[[2,1]]

2-element Vector{Int64}:
 2
 1

In [146]:
neighbors(mul(make_node!(Var, :x), add(make_node!(Constant, 1), make_node!(Constant, 1))))

x*(2)
(x*1+x*1)